In this notebook we build an "equal-weight" version of the S \& P 500 index fund.

S \& P 500 is a stock market index tracking the stock performance of 500 (actually 503...) leading companies listed on stock exchanges in the US. It is one of the most commonly followed equity indices and includes approximately 80% of the total market capitalization of US public companies, with an aggregate market cap of more than $49.8 trillion (as of March 31, 2025).

It is a capitalization-weighted index (i.e. it weight the stocks by their market capitalization) and our goal is build an equal-weighted version. In particular, smaller companies will have a larger weight than their "traditional" S \& P 500 weight and, conversly, larger companies companies will have a smaller weight.

More precisely, the goal is to create python scripts that takes the value of a given portfolio and outputs how many shares of each S \& P 500 constituent you should purchase to get an equal-weigth version of the index fund.

We start by getting the financial data using financial modeling prep api (https://site.financialmodelingprep.com).
For example, we can get some data about apple stock by doing the following simple api call

In [1]:
import requests
from my_api_key import my_api_key

api = my_api_key()
api_key = api.key
symbol = "AAPL"
api_url = f"https://financialmodelingprep.com/api/v3/quote/{symbol}?apikey={api_key}"
req = requests.get(api_url)
data = req.json()
print(data)

# For example, we can obtain today's stock price or the market capitalization of apple
price = data[0]["price"]
market_cap = data[0]["marketCap"]
print(f"stock price = {price}, market capitalization = {market_cap}")

[{'symbol': 'AAPL', 'name': 'Apple Inc.', 'price': 198.15, 'changesPercentage': 4.05945, 'change': 7.73, 'dayLow': 186.06, 'dayHigh': 199.54, 'yearHigh': 260.1, 'yearLow': 164.08, 'marketCap': 2976629115000, 'priceAvg50': 224.4894, 'priceAvg200': 228.46915, 'exchange': 'NASDAQ', 'volume': 87435915, 'avgVolume': 61669303, 'open': 186.18, 'previousClose': 190.42, 'eps': 6.97, 'pe': 28.43, 'earningsAnnouncement': '2025-05-01T20:00:00.000+0000', 'sharesOutstanding': 15022100000, 'timestamp': 1744401601}]
stock price = 198.15, market capitalization = 2976629115000


Now we retrieve (from wikipedia) the composition of the S \& P 500 and store it in a list

In [2]:
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
sp500_table = tables[0]
sp500_tickers = sp500_table['Symbol'].tolist()
sp500_tickers = [ticker.replace('.', '-') for ticker in sp500_tickers] # reformat the strings (e.g. BF.B -> BF-B)

We could loop over all S & P 500 tickers and get data into a dataframe, but it would be really inefficient. Instead, we use batch api calls

In [3]:
def chunks(input_list, chunk_size):
    for i in range(0, len(input_list), chunk_size):
        yield input_list[i: i + chunk_size]

chunked_sp500_tickers = list(chunks(sp500_tickers, chunk_size=100)) # list made up of 5 = len(sp500_tickers) // 100 lists of length 100 + 1 list of length len(sp500_tickers) % 100 
ticker_strings = [','.join(chunk) for chunk in chunked_sp500_tickers]

records = []
for ticker_string in ticker_strings:
    batch_api_call = f"https://financialmodelingprep.com/api/v3/quote/{ticker_string}?apikey={api_key}"
    req = requests.get(batch_api_call)
    
    if req.status_code != 200:
        print(f"Error fetching data for tickers: {ticker_string} (error {req.status_code})")
        continue
    
    data = req.json()
    for i, ticker in enumerate(ticker_string.split(',')):
        price = data[i]["price"]
        market_cap = data[i]["marketCap"]

        records.append({
            "Ticker": ticker,
            "Stock price": price,
            "Market cap": market_cap,
            "Strategy (nb to buy)": "N/A"
        })

sp_stocks = pd.DataFrame.from_records(records)

In [4]:
sp_stocks.head()

,Ticker,Stock price,Market cap,Strategy (nb to buy)
0,MMM,135.95,73807255000,N/A
1,AOS,64.50,9257624177,N/A
2,ABT,126.88,220050521600,N/A
3,ABBV,175.05,309659949000,N/A
4,ACN,284.34,178004232840,N/A


Given the value of the porfolio, we can now compute the number of shares to buy to have a position analogous to an "equally-weighted" version of the S \& P 500

In [5]:
import math
portfolio_size = 1000000
strategy = sp_stocks.copy()
position_size = portfolio_size / len(sp_stocks)
for i in range(len(sp_stocks.index)):
    strategy.loc[i, "Strategy (nb to buy)"] = math.floor(position_size / strategy.loc[i, "Stock price"])

In [6]:
strategy.head()

,Ticker,Stock price,Market cap,Strategy (nb to buy)
0,MMM,135.95,73807255000,14
1,AOS,64.50,9257624177,30
2,ABT,126.88,220050521600,15
3,ABBV,175.05,309659949000,11
4,ACN,284.34,178004232840,6
